This Jupyter Notebook is auxillary to the following assessment in this week.  To complete this assessment, you will complete the 5 questions outlined in this document and use the output from the Python cells to answer them.

Run the following cell to initialize your environment and begin the assessment.

In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm # loading libraries

In [2]:
path = 'nhanes_2015_2016.csv' # path file
df = pd.read_csv(path) # read the csv file

In [3]:
df.head() # dataframe preview

,SEQN,ALQ101,ALQ110,ALQ130,SMQ020,RIAGENDR,RIDAGEYR,RIDRETH1,DMDCITZN,DMDEDUC2,...,BPXSY2,BPXDI2,BMXWT,BMXHT,BMXBMI,BMXLEG,BMXARML,BMXARMC,BMXWAIST,HIQ210
0,83732,1.0,NaN,1.0,1,1,62,3,1.0,5.0,...,124.0,64.0,94.8,184.5,27.8,43.3,43.6,35.9,101.1,2.0
1,83733,1.0,NaN,6.0,1,1,53,3,2.0,3.0,...,140.0,88.0,90.4,171.4,30.8,38.0,40.0,33.2,107.9,NaN
2,83734,1.0,NaN,NaN,1,1,78,3,1.0,3.0,...,132.0,44.0,83.4,170.1,28.8,35.6,37.0,31.0,116.5,2.0
3,83735,2.0,1.0,1.0,2,2,56,3,1.0,5.0,...,134.0,68.0,109.8,160.9,42.4,38.5,37.7,38.3,110.1,2.0
4,83736,2.0,1.0,1.0,2,2,42,4,1.0,4.0,...,114.0,54.0,55.2,164.9,20.3,37.4,36.0,27.2,80.4,2.0


In [4]:
df.keys() # column names

Index(['SEQN', 'ALQ101', 'ALQ110', 'ALQ130', 'SMQ020', 'RIAGENDR', 'RIDAGEYR',
       'RIDRETH1', 'DMDCITZN', 'DMDEDUC2', 'DMDMARTL', 'DMDHHSIZ', 'WTINT2YR',
       'SDMVPSU', 'SDMVSTRA', 'INDFMPIR', 'BPXSY1', 'BPXDI1', 'BPXSY2',
       'BPXDI2', 'BMXWT', 'BMXHT', 'BMXBMI', 'BMXLEG', 'BMXARML', 'BMXARMC',
       'BMXWAIST', 'HIQ210'],
      dtype='object')

In [5]:
variables = ["BPXSY1", "RIDAGEYR", "RIAGENDR", "RIDRETH1", 
              "DMDEDUC2", "BMXBMI", "SMQ020", "SDMVSTRA", "SDMVPSU"]
df = df[variables].dropna() # drop unused variables and missing values

In [6]:
df['group'] = 10*df.SDMVSTRA + df.SDMVPSU # define clusters

In [7]:
df['smq'] = df.SMQ020.replace({2: 0, 7: np.nan, 9: np.nan}) # recode smoking variable

#### Question 1: What is clustered data?

Is a **data structure where observations are grouped into clusters**. Where observations within the same cluster tend to be more similar (or *correlated*) to each other than to those in other clusters.

#### Question 2: Which of the following features has the highest correlation between two observations in the same cluster?

SDMVSTRA has the highest correlation between two observations in the same cluster, as expected, since **SDMVSTRA is directly related to the way clusters are defined**.

In [8]:
np.random.seed(123)

In [10]:
for variable in ["BPXSY1", "SDMVSTRA", "RIDAGEYR", "BMXBMI", "smq"]:
    gee_model = sm.GEE.from_formula(variable + ' ~ 1', groups = 'group', 
                                    cov_struct=sm.cov_struct.Exchangeable(), data = df) # define a marginal model using GEE approach
    result = gee_model.fit() # estimate model parameters
    print(variable, result.cov_struct.summary())

BPXSY1 The correlation between two observations in the same cluster is 0.030
SDMVSTRA The correlation between two observations in the same cluster is 0.959
RIDAGEYR The correlation between two observations in the same cluster is 0.035
BMXBMI The correlation between two observations in the same cluster is 0.039
smq The correlation between two observations in the same cluster is 0.026


#### Question 3: What is true about multiple linear regression and marginal linear models when dependence is present in data?

Marginal linear model estimates and standard errors are meaningful due to the dependence of data, but only when dependence is strictly between observations within the same group.

#### Question 4: Multilevel models are expressed in terms of _____.

Random effects.

#### Question 5:  Which of the following is true regarding reasons why we fit marginal models?

- Quicker computational times, faster estimation
- Robust standard errors that reflect the specified correlation structure
- Easier accommodation of non-normal outcomes